# Hypothesis Testing

In this notebook, we test the hypotheses defined in 01_hypothesis_design using classical statistical tests. The goal is to determine whether differences in ride demand across weekdays/weekends and peak/off-peak hours are statistically significant.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

AGG_PATH = "../../data/processed/nyc_demand_zone_hour_2019_q1.parquet"

In [2]:
# Show all columns
pd.set_option("display.max_columns", None)

# Load aggregated dataset
df = pd.read_parquet(AGG_PATH)
df.head()

,zone_id,pickup_hour_ts,demand,avg_fare,avg_distance,hour,day_of_week,is_weekend,day,month
0,1,2019-01-01 10:00:00,2,61.25,16.90,10,1,0,1,1
1,1,2019-01-01 12:00:00,1,135.00,19.30,12,1,0,1,1
2,1,2019-01-01 15:00:00,1,106.00,41.28,15,1,0,1,1
3,1,2019-01-02 02:00:00,1,30.00,1.27,2,2,0,2,1
4,1,2019-01-02 03:00:00,1,15.00,12.65,3,2,0,2,1


## Weekday vs Weekend
Perform an independent two-sample t-test (Welch’s t-test) because:
- observations are large in number
- variances between groups may differ
- data is skewed, but with large n the test is robust

Hypothesis:
- H0: mean zone-hour demand is the same for weekdays and weekends
- H1: mean zone-hour demand differs between weekdays and weekends

In [3]:
weekday = df[df["is_weekend"] == 0]["demand"].values
weekend = df[df["is_weekend"] == 1]["demand"].values

# Two-sample t-test
t_stat, p_value = stats.ttest_ind(weekday, weekend, equal_var=False)

t_stat, p_value

(np.float64(7.49199188495313), np.float64(6.81558301004871e-14))

p < 0.05 -> demand differs on weekends and weekdays

## Peak vs Off-Peak
Test whether mean demand during peak hours is significantly higher than during off-peak hours using Welch’s t-test.

Peak time interval: 17:00 - 19:00 (the rides starting at 17:00 and 18:00)

In [4]:
peak_hours = [17, 18]
peak = df[df["hour"].isin(peak_hours)]["demand"].values
offpeak = df[~df["hour"].isin(peak_hours)]["demand"].values

t_stat_peak, p_value_peak = stats.ttest_ind(peak, offpeak, equal_var=False)
t_stat_peak, p_value_peak

(np.float64(32.86124589747304), np.float64(1.932349577076067e-232))

p < 0.05 -> peak demand is significantly higher

## Assumptions of the t-test

- Observations are approximately independent (zone-hour)
- Distribution of the sample means is approximately normal (CLT)
- Variance may differ between groups (Welch’s t-test)
- Large sample size makes the test robust to skewness